In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime, timedelta
import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('H') # - timedelta(hours=1)
print(f'{current_date=}')
# current_date = pd.Timestamp('2023-02-28 09:00:00')

current_date=Timestamp('2023-11-10 21:00:00')


In [3]:
from src.helpers.inference import load_batch_of_features_from_store

features = load_batch_of_features_from_store(current_date)


Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/178327
Connected. Call `.close()` to terminate connection gracefully.
2023-11-10 22:25:59,300 INFO: Feature view already exists, skipping creation.
Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/178327
Connected. Call `.close()` to terminate connection gracefully.
Finished: Reading data from Hopsworks, using ArrowFlight (8.82s) from Hopsworks, using ArrowFlight.   Reading data from Hopsworks, using ArrowFlight...   Reading data from Hopsworks, using ArrowFlight.   Reading data from Hopsworks, using ArrowFlight...   Reading data from Hopsworks, using ArrowFlight.   Reading data from Hopsworks, using ArrowFlight...   Reading data from Hopsworks, using ArrowFlight.   


In [ ]:
from src.helpers.inference import (
    load_model_from_registry,
    get_model_predictions
)

model = load_model_from_registry()
predictions = get_model_predictions(model, features)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/178327
Connected. Call `.close()` to terminate connection gracefully.


In [5]:
predictions['pickup_hour'] = current_date
predictions

NameError: name 'predictions' is not defined

In [6]:
from src.feature_store_api import get_feature_store
import src.config as config

# connect to the feature group
feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS,
    version=1,
    description="Predictions generate by our production model",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/178327
Connected. Call `.close()` to terminate connection gracefully.


In [7]:
feature_group.insert(predictions, write_options={"wait_for_job": False})


NameError: name 'predictions' is not defined